# Inference

Inference on the simulation parameters is carried out employing Sequential Neural Posterior Estimation (SNPE), implemented by the `sbi` (acronym for simulation based inference) package. This notebooks explores the code behind the process of building a posterior distribution and get samples from it.

### A. Imports and preliminary steps

First of all, the relevant imports are carried out in the cell below.

In [1]:
%load_ext autoreload
%autoreload 2

# Just a formatting related plugin
%load_ext nb_black

%matplotlib inline
import matplotlib.pyplot as plt

import sys

sys.path.append("../")

import multiprocessing as mp

from collections import deque
from pathlib import Path
from typing import Dict, Optional

import arviz
import pickle

import numpy as np
import pandas as pd
import pyreadr
import sbi
import sbi.utils as sbi_utils
import seaborn as sns
import statsmodels.formula.api as smf
import torch

from joblib import Parallel, delayed
from matplotlib.lines import Line2D
from scipy.stats import ttest_ind
from snpe.inference import inference_class
from snpe.simulations import simulator_class, marketplace_simulator_class
from snpe.embeddings.embeddings_to_ratings import EmbeddingRatingPredictor
from snpe.utils.statistics import review_histogram_correlation
from snpe.utils.tqdm_utils import tqdm_joblib
from tqdm import tqdm

# Set plotting parameters
sns.set(style="white", context="talk", font_scale=2.5)
sns.set_color_codes(palette="colorblind")
sns.set_style("ticks", {"axes.linewidth": 2.0})

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<IPython.core.display.Javascript object>

Then the path where the inference output is going to be stored, as well as where the relevant inputs for the functions are located is defined.

In [2]:
ARTIFACT_PATH = Path("../../../")

<IPython.core.display.Javascript object>

### B. Building a posterior distribution

The function in the code cell below is designed to carry out inference on the basis of time series simulation output.

The first lines of the function are devoted to initialize two tensors containing the upper and lower bounds for the parameter distributions, set the device for the training of the inference model and finally instantiate the inferrer, which in this case is an object of the `TimeSeriesInference` class (See `inference_class.py`).

Once this is done, the `load_simulator` method of the inferrer is called. This method loads a simulator of the appropriate type (as specified when invoking the function) which will subsequently load a simulation output to be fed to the inference model. Then the inferrer's method `infer_snpe_posterior` is called, this method will:

   1. Create an embedding net employing a subset of the simulations.
   2. Employ the embedding net to obtain a function that will build a density estimator for learning the posterior (posterior net)
   3. Conduct SNPE  with the posterior net and the prior bounds provided.
   4. Builds a posterior from the neural density estimator.
    
Finally the method `save_inference` stores posterior in a `.pkl` file alongside other relevant pieces of information about the simulation and the inference procedure.

The arguments to provide to the function are the following:

- `device`: Device where the inference excercise will take place. Must be a string, either 'cuda' or 'cpu'.
- `simulator_type`: Type of the simulator to be loaded by the inferrer (e.g. double_herding, marketplace...).
- `simulation_type`: Modality of simulation that will be loaded. Must be str 'timeseries' or 'histogram'.
- `params`: Dictionary of parameters to provide to the embedding net creator.

In [3]:
def infer_and_save_posterior(
    device: str, simulator_type: str, simulation_type: str, params: Dict
) -> None:
    parameter_prior = sbi_utils.BoxUniform(
        low=torch.tensor([0.0, 0.0, 0.0, 0.0, 0.5, 0.25, 0.25, 0.5]).type(
            torch.FloatTensor
        ),
        high=torch.tensor([4.0, 4.0, 1.0, 1.0, 1.0, 0.75, 0.75, 1.0]).type(
            torch.FloatTensor
        ),
        device=device,
    )
    inferrer = inference_class.TimeSeriesInference(
        parameter_prior=parameter_prior, device=device
    )
    inferrer.load_simulator(
        dirname=ARTIFACT_PATH,
        simulator_type=simulator_type,
        simulation_type=simulation_type,
    )
    batch_size = params.pop("batch_size")
    learning_rate = params.pop("learning_rate")
    hidden_features = params.pop("hidden_features")
    num_transforms = params.pop("num_transforms")
    inferrer.infer_snpe_posterior(
        embedding_net_conf=params,
        batch_size=batch_size,
        learning_rate=learning_rate,
        hidden_features=hidden_features,
        num_transforms=num_transforms,
    )
    inferrer.save_inference(ARTIFACT_PATH)

<IPython.core.display.Javascript object>

The following cell defines the dictionary of hyperparameters for the inference model that has to be provided as argument to `infer_and_save_posterior` to shape the embedding net. This particular set of values have been obtained by carrying out a hyperparameter optimization procedure.



In [4]:
# diccionario de parámetros requerido para utilizar la función

inference_params = {
    "batch_size": 64,
    "learning_rate": 3.1e-4,
    "hidden_features": 70,
    "num_transforms": 8,
    "num_conv_layers": 2,
    "num_channels": 9,
    "conv_kernel_size": 17,
    "maxpool_kernel_size": 11,
    "num_dense_layers": 2,
}

<IPython.core.display.Javascript object>

Now, with all the ingredients in place, the `infer_and_save_posterior` function can be called.

In [5]:
infer_and_save_posterior("cuda", "marketplace", "timeseries", inference_params)

/opt/conda/lib/python3.7/site-packages/sbi/utils/torchutils.py:28: UserWarning: GPU was selected as a device for training the neural network. Note that we expect **no** significant speed ups in training for the default architectures we provide. Using the GPU will be effective only for large neural networks with operations that are fast on the GPU, e.g., for a CNN or RNN `embedding_net`.
  "GPU was selected as a device for training the neural network. "


Embedding net created: 
 Sequential(
  (0): Conv1d(5, 9, kernel_size=(17,), stride=(1,), padding=(8,))
  (1): LeakyReLU(negative_slope=0.01)
  (2): Conv1d(9, 9, kernel_size=(17,), stride=(1,), padding=(16,), dilation=(2,))
  (3): MaxPool1d(kernel_size=11, stride=11, padding=0, dilation=1, ceil_mode=False)
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=459, out_features=64, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=64, out_features=32, bias=True)
)


/opt/conda/lib/python3.7/site-packages/sbi/utils/user_input_checks.py:697: UserWarning: Parameters theta has device 'cpu'. Moving theta to the data_device 'cuda:0'.Training will proceed on device 'cuda:0'.
  f"Parameters theta has device '{theta.device}'. "


 Neural network successfully converged after 127 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 127
        Best validation performance: 3.0781
        -------------------------
        


<IPython.core.display.Javascript object>

### C. Sampling from the posterior

The following function `sample_posterior_with_observed` gets parameter samples form the posterior distribution built by `infer_and_save_posterior` conditioned on the observations provided.

- `device`: Device where the sampling operation will take place. Must a string, either 'cuda' or 'cpu'.
- `simulator_type`: Type of the simulator to be loaded by the inferrer instance in charge of drawing the samples (e.g. double_herding, marketplace...).
- `simulation_type`: Modality of simulation that will be loaded. Must be str 'timeseries' or 'histogram'.
- `num_samples`: Number of samples to take from the defined posterior.
- `observations`: Simulation data for which the inferrer will samples. 

The function returns an numpy array of the shape (num_samples, < number_of_products_in_observations >, < number_of_parameters >) with the sampled posteriors.

In [6]:
def sample_posterior_with_observed(
    device: str,
    observations: np.array,
    num_samples: int,
    simulator_type: str,
    simulation_type: str,
) -> np.ndarray:
    # The parameter prior doesn't matter here as it will be overridden by that of the loaded inference object
    parameter_prior = sbi.utils.BoxUniform(
        low=torch.tensor([0.0, 0.0, 0.0, 0.0, 0.5, 0.25, 0.25, 0.5]).type(
            torch.FloatTensor
        ),
        high=torch.tensor([4.0, 4.0, 1.0, 1.0, 1.0, 0.75, 0.75, 1.0]).type(
            torch.FloatTensor
        ),
        device=device,
    )
    inferrer = inference_class.TimeSeriesInference(
        parameter_prior=parameter_prior, device=device
    )
    inferrer.load_simulator(
        dirname=ARTIFACT_PATH,
        simulator_type=simulator_type,
        simulation_type=simulation_type,
    )
    inferrer.load_inference(dirname=ARTIFACT_PATH)
    posterior_samples = inferrer.get_posterior_samples(
        observations, num_samples=num_samples
    )
    return posterior_samples

<IPython.core.display.Javascript object>

In the example below, `sample_posterior_with_observed` is set to get 10 parameter samples conditioned on a marketplace timeseries simulation (`timeseries_data`) which is imported immediately before actually calling the function.

In [10]:
timeseries_data = np.load(ARTIFACT_PATH / 'timeseries_data.npy')

posterior_samples = sample_posterior_with_observed("cuda", timeseries_data, 10, "marketplace", "timeseries")